In [218]:
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
from pyproj import Proj, Transformer

In [219]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [220]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['mother_road']='start'

buildings['road_direction'] = 'None'
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [221]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def addDirection_BuildingCode(direction,length,index,road_index):
    #right-even, left-odd number 
    length=int(length)
    # print(length)
    if length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}

        gdf = gpd.GeoDataFrame(d, crs=4326)
        sp_point=gdf.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                my_line=l
                break

        result=split(my_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            break
        splitted_line.srid = 4326
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
        buildings.at[index,'road_length_d_code']=str(int(geom_transformed.length))+"/"
    if (direction == 1):
        # print("Right Direction")
        buildings.at[index,'road_direction']="right"
        buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(length)))
        
    elif (direction == -1):
        # print("Left Direction")
        buildings.at[index,'road_direction']="left"
        buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(length)))
    else:
        # print("On  Line")

        buildings.at[index,'road_direction']="on_line"



            
            
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(x1,y1,x2,y2,bearing,index):
    # default direction is west to east , south to north 
    if bearing is None :
        bearing=0
    if (bearing >= 315 and bearing<=45) or (bearing >= 135 and bearing<=225) :
    #    North_South/south_north Direction, assign to south
        pass

     
    else :
        # east_west/west_east direction , assign to east
        road.at[index,'start_point_x']=float(x2)
        road.at[index,'start_point_y']=float(y2)
        road.at[index,'end_point_x']=float(x1)
        road.at[index,'end_point_y']=float(y1)
        road.at[index,'mother_road']='end'
            
    

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            status=False
            if line.distance(start_point) < 1e-8 :
                status=True
                # print("start found "+ str(row.NEW))
                road.at[index_road,'start']=str(row.NEW)
            if line.distance(end_point) < 1e-8:
                # print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)
                if status == False:
                    #only endpoint found
                    road.at[index_road,'start_point_x']=float(end_point.x)
                    road.at[index_road,'start_point_y']=float(end_point.y)
                    road.at[index_road,'end_point_x']=float(start_point.x)
                    road.at[index_road,'end_point_y']=float(start_point.y)
                    road.at[index_road,'mother_road']='end'
                    
                else:
                    #both start and end exist which means line is in between roads 
                    ReviseStartEndDateWithBearing(start_point.x,start_point.y,end_point.x,end_point.y,line_row.bearing,index_road)
                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def CalculateTouchLine():
    for index,row in road.iterrows():
        # print(row)
        start_point,end_point=getStartEndPoint(row,"road")
        addStartEndPoint(start_point,end_point,index)
        CalculateBearing(start_point,end_point,index)
        FindIntersectedFeatures(start_point,end_point,row,index)
        # break
    print("Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line , bearing ")

def assign_building_code():
    road['start_point_x'].apply(lambda x: float(x))
    road['start_point_y'].apply(lambda x: float(x))
    road['end_point_x'].apply(lambda x: float(x))
    road['end_point_y'].apply(lambda x: float(x))
    
    
    
    for index, row in buildings.iterrows():
        road_code = row.NEW
        road_linked_wrt_point= road.loc[road['NEW'] == road_code]
        length= getLineLength(road_linked_wrt_point)
        try:
            road_index=road_linked_wrt_point.index[0]
            road.at[road_index,'length']=length[0]       
            start = bdcPoint()
            end = bdcPoint()
            point = bdcPoint()
            poi = row.geometry
            # print(type(road_linked_wrt_point.start_point_x))
            # try:
            #     start.x,start.y,end.x,end.y,point.x,point.y=float(road_linked_wrt_point.start_point_x),float(road_linked_wrt_point.start_point_y),float(road_linked_wrt_point.end_point_x),float(road_linked_wrt_point.end_point_y),poi.x,poi.y
            # except:
            try:
                start.x,start.y,end.x,end.y,point.x,point.y=float(road_linked_wrt_point.start_point_x.astype('float', copy=False)),float(road_linked_wrt_point.start_point_y.astype('float', copy=False)),float(road_linked_wrt_point.end_point_x.astype('float', copy=False)),float(road_linked_wrt_point.end_point_y.astype('float', copy=False)),poi.x,poi.y
                    
                direction=directionOfPoint(start,end,point)
                # print(direction)
                addDirection_BuildingCode(direction,length[0],index,road_index)
            except:
                continue
        except:
            continue
        
        # break
        
    print("-------Sucess ------ added: road_length_d_code , length of road , direction of building ")

    
                   


In [222]:
CalculateTouchLine()

/tmp/ipykernel_14858/219173717.py:108: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  start_point,end_point=df_line[0],df_line[1]


Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line , bearing 


In [223]:
assign_building_code()     

/tmp/ipykernel_14858/219173717.py:70: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for l in line:
/tmp/ipykernel_14858/219173717.py:75: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for r  in result:
/tmp/ipykernel_14858/219173717.py:78: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  splitted_line.srid = 4326
/tmp/ipykernel_14858/219173717.py:70: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for l in line:
/tmp/ipykernel_14858/219173717.py:75: S

-------Sucess ------ added: road_length_d_code , length of road , direction of building 


In [224]:
buildings.to_file("../app_outputs/output_building.geojson", driver="GeoJSON")

In [225]:
road.to_file("../app_outputs/output_road.geojson", driver="GeoJSON")